In [2]:
import re
import requests
from bs4 import BeautifulSoup as bs

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

pd.set_option('notebook_repr_html', True)

In [4]:
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
league = ['GB1','FR1','L1','IT1','ES1']
league_page = "https://www.transfermarkt.com/jumplist/startseite/wettbewerb/"

In [5]:
def get_club_details(tr_tag):
    club = tr_tag.find_all('a',class_="vereinprofil_tooltip")[1]
    club_link = club['href']
    club_name = club.get_text()
    club_value = tr_tag.find_all('td',class_="rechts show-for-small show-for-pad nowrap")[0].get_text()
    return tuple((club_link,club_name,club_value))

In [6]:
clubs_list = []
for league_id in league:
    page = requests.get(league_page + league_id,headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('tbody')[1]
    tr_container = tbody_container.find_all('tr')
    for tr_tag in tr_container :
        clubs_list.append(get_club_details(tr_tag))
print('All the clubs were uploaded')

All the club were uploaded


In [7]:
page = requests.get(league_page + 'GB1',headers = headers)
soup = bs(page.content, 'html.parser')

In [8]:
soup

<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<script type="text/javascript">
    !function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.document, r = !!n.frames[t]; if (!r) if (a.body) { var i = a.createElement("iframe"); i.style.cssText = "display:none", i.name = t, a.body.appendChild(i) } else setTimeout(e, 5); return !r }(), n.__tcfapi = function () { for (var e, t = arguments.length, n = new Array(t), r = 0; r < t; r++)n[r] = arguments[r]; if (!n.length) return a; if ("setGdprApplies" === n[0]) n.length > 3 && 2 === parseInt(n[1], 10) && "boolean" == typeof n[3] && (e = n[3], "function" == typeof n[2] && n[2]("set", !0)); else if ("ping" === n[0]) { var i = { gdprApplies: e, cmpLoaded: !1, cmpStatus: "stub" }; "function" == typeof n[2] && n[2](i) } else a.push(n) }, n.addEventListener("message", (f

In [9]:
tbody_container = soup.find_all('tbody')[1]

In [10]:
tbody_container

<tbody>
<tr class="odd">
<td class="zentriert no-border-rechts"><a class="vereinprofil_tooltip" href="/manchester-city/startseite/verein/281/saison_id/2021"><img alt="Manchester City" class="tiny_wappen" src="https://tmssl.akamaized.net/images/wappen/tiny/281.png?lm=1467356331" title=" "/></a></td><td class="hauptlink no-border-links hide-for-small hide-for-pad"><a class="vereinprofil_tooltip" href="/manchester-city/startseite/verein/281/saison_id/2021" id="281">Manchester City</a> <a href="#"><img alt="English Champion 20/21" class="tabelle-erfolg" src="https://tmssl.akamaized.net/images/erfolge/mini/12.png?lm=1520606997" title="English Champion 20/21"/></a></td><td class="hauptlink no-border-links show-for-small show-for-pad"><a class="vereinprofil_tooltip" href="/manchester-city/startseite/verein/281/saison_id/2021" id="281">Man City</a></td><td class="zentriert"><a href="/manchester-city/kader/verein/281/saison_id/2021" title="Manchester City">23</a></td><td class="zentriert hide-f

In [11]:
tr_container = tbody_container.find_all('tr')

In [12]:
tr_container

[<tr class="odd">
 <td class="zentriert no-border-rechts"><a class="vereinprofil_tooltip" href="/manchester-city/startseite/verein/281/saison_id/2021"><img alt="Manchester City" class="tiny_wappen" src="https://tmssl.akamaized.net/images/wappen/tiny/281.png?lm=1467356331" title=" "/></a></td><td class="hauptlink no-border-links hide-for-small hide-for-pad"><a class="vereinprofil_tooltip" href="/manchester-city/startseite/verein/281/saison_id/2021" id="281">Manchester City</a> <a href="#"><img alt="English Champion 20/21" class="tabelle-erfolg" src="https://tmssl.akamaized.net/images/erfolge/mini/12.png?lm=1520606997" title="English Champion 20/21"/></a></td><td class="hauptlink no-border-links show-for-small show-for-pad"><a class="vereinprofil_tooltip" href="/manchester-city/startseite/verein/281/saison_id/2021" id="281">Man City</a></td><td class="zentriert"><a href="/manchester-city/kader/verein/281/saison_id/2021" title="Manchester City">23</a></td><td class="zentriert hide-for-sma

In [13]:
for tr_tag in tr_container:
    club = tr_tag.find_all('a',class_="vereinprofil_tooltip")[1]
    club_link = club['href']
    print(club_link)
    # club_name = club.get_text()
    # club_value = tr_tag.find_all('td',class_="rechts show-for-small show-for-pad nowrap")[0].get_text()

/manchester-city/startseite/verein/281/saison_id/2021
/manchester-united/startseite/verein/985/saison_id/2021
/fc-chelsea/startseite/verein/631/saison_id/2021
/fc-liverpool/startseite/verein/31/saison_id/2021
/tottenham-hotspur/startseite/verein/148/saison_id/2021
/fc-arsenal/startseite/verein/11/saison_id/2021
/leicester-city/startseite/verein/1003/saison_id/2021
/fc-everton/startseite/verein/29/saison_id/2021
/aston-villa/startseite/verein/405/saison_id/2021
/wolverhampton-wanderers/startseite/verein/543/saison_id/2021
/west-ham-united/startseite/verein/379/saison_id/2021
/leeds-united/startseite/verein/399/saison_id/2021
/brighton-amp-hove-albion/startseite/verein/1237/saison_id/2021
/fc-southampton/startseite/verein/180/saison_id/2021
/crystal-palace/startseite/verein/873/saison_id/2021
/newcastle-united/startseite/verein/762/saison_id/2021
/fc-brentford/startseite/verein/1148/saison_id/2021
/norwich-city/startseite/verein/1123/saison_id/2021
/fc-watford/startseite/verein/1010/sais

In [14]:
clubs_list

[('/manchester-city/startseite/verein/281/saison_id/2021',
  'Manchester City',
  '€46.84m'),
 ('/manchester-united/startseite/verein/985/saison_id/2021',
  'Manchester United',
  '€31.42m'),
 ('/fc-chelsea/startseite/verein/631/saison_id/2021', 'Chelsea FC', '€33.02m'),
 ('/fc-liverpool/startseite/verein/31/saison_id/2021',
  'Liverpool FC',
  '€32.09m'),
 ('/tottenham-hotspur/startseite/verein/148/saison_id/2021',
  'Tottenham Hotspur',
  '€29.04m'),
 ('/fc-arsenal/startseite/verein/11/saison_id/2021', 'Arsenal FC', '€21.67m'),
 ('/leicester-city/startseite/verein/1003/saison_id/2021',
  'Leicester City',
  '€19.58m'),
 ('/fc-everton/startseite/verein/29/saison_id/2021', 'Everton FC', '€16.49m'),
 ('/aston-villa/startseite/verein/405/saison_id/2021',
  'Aston Villa',
  '€17.20m'),
 ('/wolverhampton-wanderers/startseite/verein/543/saison_id/2021',
  'Wolverhampton Wanderers',
  '€16.00m'),
 ('/west-ham-united/startseite/verein/379/saison_id/2021',
  'West Ham United',
  '€14.76m'),
 (

In [15]:
def get_players_club(player):
    player_id = player['id']
    player_link = player['href']
    player_name = player.get_text()
    return tuple((player_id,player_link,player_name,club_name,club_value))

In [16]:
url_site = "https://www.transfermarkt.com"
player_list = []
for club_link,club_name,club_value in clubs_list:
    page = requests.get(url_site + club_link,headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('tbody')[1]
    players_details = tbody_container.find_all('a',class_="spielprofil_tooltip")
    for player in players_details[::2] :
        player_list.append(get_players_club(player))
print('All the players were uploaded')

All the players were uploaded


In [17]:
len(player_list)

2703

In [18]:
def get_profil_detail(soup):
    table_container = soup.find_all('table', class_="auflistung")[0]
    td_container = table_container.find_all('td')
    if td_container[1].find('a') == None:
        birth = td_container[0].find('a')['href'].split("/")[-1]# take it in format YYYY-MM-DD for datetime later
        height = td_container[3].get_text().split("m")[0]# I remove the m of meter 
        country = td_container[4].find('img')['title']
        role = td_container[5].get_text().strip()
        foot = td_container[6].get_text()
    else :
        birth = td_container[1].find('a')['href'].split("/")[-1]
        height = td_container[4].get_text().split("m")[0]# I remove the m of meter 
        country = td_container[5].find('img')['title']
        role = td_container[6].get_text().strip()
        foot = td_container[7].get_text()
    tbody_container = soup.find_all('tbody')[0]
    tr_transfer_container = tbody_container.find_all('tr',class_="zeile-transfer")
    transfer_list = []
    for tr_transfer_tag in tr_transfer_container:
        td_transfer_container = tr_transfer_tag.find_all("td")
        tranfer_from = td_transfer_container[5].get_text()
        transfer_to = td_transfer_container[9].get_text()
        transfer_season = td_transfer_container[0].get_text()
        transfer_date = td_transfer_container[1].get_text()
        transfer_list.append(tuple((tranfer_from,transfer_to,transfer_season,transfer_date)))
    return tuple((Id,name,club,club_value,birth,height,country,role,foot,link.split("/")[1],transfer_list))

In [19]:
player_details = []
i=0
for Id,link,name,club,club_value in player_list:
    i=i+1
    if i%500 == 0:
        print("new league upload")
    try:
        page = requests.get(url_site + link,headers = headers)
        soup = bs(page.content, 'html.parser')
        player_details.append(get_profil_detail(soup))
    except Exception as e:
        player_details.append(tuple((Id,name,club,club_value,None,None,None,None,None,link.split("/")[1],[])))
        continue
print("all player details uploaded")

new league upload
new league upload
new league upload
new league upload
new league upload
all player details uploaded


In [20]:
# Saving player_details as pickle file
import pickle

with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_details.pickle', 'wb') as handle:
    pickle.dump(player_details, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [23]:
len(player_details)

2703

In [21]:
# Reading player_details as pickle file

with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_details.pickle', 'rb') as handle:
    pickle_player_details = pickle.load(handle)

In [22]:
len(pickle_player_details)

2703

In [ ]:
def get_injuries_details(soup):
    tbody_container = soup.find_all('tbody')[0]
    tr_container = tbody_container.find_all('tr')
    injuries_list = []
    for tr_tag in tr_container:
        season = tr_tag.find_all('td')[0]
        injury = tr_tag.find_all('td')[1]
        time_out = tr_tag.find_all('td')[4]
        injuries_list.append(tuple((season.get_text(),injury.get_text(),time_out.get_text().split()[0])))
    return injuries_list

In [ ]:
player_list = []
for Id,name,club,club_value,birth,height,country,role,foot,name_link,transfer_list in player_details:
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
        page = requests.get("https://www.transfermarkt.com/{}/verletzungen/spieler/{}".format(name_link,Id),headers=headers)
        soup = bs(page.content, 'html.parser')
        player_list.append(tuple((Id,name,club,club_value,birth,height,country,role,foot,transfer_list,get_injuries_details(soup))))
    except Exception as e:
        player_list.append(tuple((Id,name,club,club_value,birth,height,country,role,foot,transfer_list,[])))
        continue
print("all player injuries details uploaded")
print("End of uploading from Transfermarkt")

all player injuries details uploaded
End of uploading from Transfermarkt


In [ ]:
pd.DataFrame(player_list)

,0,1,2,3,4,5,6,7,8,9,10
0,238223,Ederson,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Virus Infection, 16), (19/20, Ill, 8)..."
1,221624,Zack Steffen,Manchester City,€45.75m,None,None,None,None,None,[],"[(21/22, Corona virus, 16), (20/21, Bone Bruis..."
2,14555,Scott Carson,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Virus Infection, 67), (12/13, Unknown..."
3,258004,Rúben Dias,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Knock, 4), (17/18, Ankle Injury, 11),..."
4,176553,Aymeric Laporte,Manchester City,€45.75m,None,None,None,None,None,[],"[(21/22, Fitness, 8), (20/21, Strain, 21), (20..."
...,...,...,...,...,...,...,...,...,...,...,...
2686,131275,Choco Lozano,Cádiz CF,€1.83m,None,None,None,None,None,[],"[(20/21, Corona virus, 15), (17/18, Minor Knoc..."
2687,177065,Florin Andone,Cádiz CF,€1.83m,None,None,None,None,None,[],"[(19/20, Ruptured cruciate ligament, 272), (19..."
2688,85383,Rubén Sobrino,Cádiz CF,€1.83m,None,None,None,None,None,[],"[(19/20, Muscular problems, 12), (19/20, Leg I..."
2689,18644,Álvaro Negredo,Cádiz CF,€1.83m,None,None,None,None,None,[],"[(20/21, Leg Injury, 13), (17/18, Unknown Inju..."


In [ ]:
def get_list_club_lequipe(h2_tag,league_id):
    if league_id == 'EQ_D1' :
        club = h2_tag.find_all('a')[1]
        club_link = club['href']
        club_name = club['title']
        return tuple((club_link,club_name))
    else:
        club = h2_tag.find('a')
        club_link = club['href']
        club_name = club['title']
        return tuple((club_link,club_name))

In [ ]:
league = ['EQ_ANG','EQ_D1','EQ_ALL','EQ_ITA','EQ_ESP']
league_page = "https://www.lequipe.fr/Football/"
clubs_list_lequipe = []
for league_id in league:
    print(league_id)
    page = requests.get(league_page + league_id + ".html",headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('div',class_="listeclubs")[0]
    h2_container = tbody_container.find_all('h2')
    for h2_tag in h2_container :
        clubs_list_lequipe.append(get_list_club_lequipe(h2_tag,league_id))
print('All the club were uploaded')

EQ_ANG
EQ_D1


IndexError: ignored

In [ ]:
url_site = "https://www.lequipe.fr"
player_list_lequipe = []
for club_link,club_name in clubs_list_lequipe:
    page = requests.get(url_site + club_link,headers = headers)
    soup = bs(page.content, 'html.parser')
    tbody_container = soup.find_all('table')[4]
    players_details = tbody_container.find_all('tr')
    for player in players_details[1:] :
        player_details = player.find_all('a')[0]
        weight = player.find_all('td')[6]
        player_name = player_details.get_text().strip()
        player_list_lequipe.append(tuple((player_name,club_name,weight.get_text())))
print("all player from lequipe were uploaded")

In [ ]:
club_to_player_map = {}
for player in player_list_lequipe:
    if player[1] in club_to_player_map.keys():
        club_to_player_map[player[1]][player[0]] = player[2]
    else:
        club_to_player_map[player[1]] = {}
        club_to_player_map[player[1]][player[0]] = player[2]

In [ ]:
#Just to verify the name
lequipe_clubs = club_to_player_map.keys()
print(lequipe_clubs)

In [ ]:
import difflib

In [ ]:
df_players = pd.DataFrame(player_list)
df_players.head()

,0,1,2,3,4,5,6,7,8,9,10
0,238223,Ederson,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Virus Infection, 16), (19/20, Ill, 8)..."
1,221624,Zack Steffen,Manchester City,€45.75m,None,None,None,None,None,[],"[(21/22, Corona virus, 16), (20/21, Bone Bruis..."
2,14555,Scott Carson,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Virus Infection, 67), (12/13, Unknown..."
3,258004,Rúben Dias,Manchester City,€45.75m,None,None,None,None,None,[],"[(20/21, Knock, 4), (17/18, Ankle Injury, 11),..."
4,176553,Aymeric Laporte,Manchester City,€45.75m,None,None,None,None,None,[],"[(21/22, Fitness, 8), (20/21, Strain, 21), (20..."


In [ ]:
Final_player = []
for player_transfertmarkt in player_list:
    try:
        list_of_players_lequipe = club_to_player_map[difflib.get_close_matches(player_transfertmarkt[2], lequipe_clubs,cutoff=0.1)[0]].keys()
        weight_player = club_to_player_map[difflib.get_close_matches(player_transfertmarkt[2], lequipe_clubs,cutoff=0.1)[0]][difflib.get_close_matches(player_transfertmarkt[1][:15], list_of_players_lequipe,cutoff=0.1)[0]]
        Final_player.append(tuple((player_transfertmarkt[0],player_transfertmarkt[1],player_transfertmarkt[2],player_transfertmarkt[3],player_transfertmarkt[4],weight_player,player_transfertmarkt[5],player_transfertmarkt[6],player_transfertmarkt[7],player_transfertmarkt[8],player_transfertmarkt[9],player_transfertmarkt[10])))
    except Exception as e:
        print(player_transfertmarkt[2])
        Final_player.append(tuple((player_transfertmarkt[0],player_transfertmarkt[1],player_transfertmarkt[2],player_transfertmarkt[3],player_transfertmarkt[4],None,player_transfertmarkt[5],player_transfertmarkt[6],player_transfertmarkt[7],player_transfertmarkt[8],player_transfertmarkt[9],player_transfertmarkt[10])))
print("END")
print(Final_player)

Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester City
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Manchester United
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea FC
Chelsea

In [ ]:
df_player = pd.DataFrame(Final_player)
df_player.columns = ["id", "name", "club","club_value","birth","height","weight","country","role","foot","transfers","injuries"]
df_player.head()
df_player.shape

(2691, 12)

In [ ]:
df_player[df_player['name'] == 'Figo']

,id,name,club,club_value,birth,height,weight,country,role,foot,transfers,injuries
